In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "huangtuoyue/GPT2-GOT2-finetuned"

generator = pipeline("text-generation", model=model_name, tokenizer=model_name,\
    do_sample=True, max_length=100, temperature=1, top_p = 1, pad_token_id = 50256, repetition_penalty=1)

output = generator("Once upon a time")

In [17]:
output = generator("Winter is coming. ")
print(output)

[{'generated_text': 'Winter is coming.Winter is coming. The Stark words echo all around us, but Jon remembered the words of the last night he spent with the brothers. He ’ d felt the tears on his cheeks like blueberries on a melon. The bittersweet farewells of the two companions he ’ d spent so much time with. The silent promises of the evening. He remembered how he ’ d pleaded to sleep beside them, how he woke up in the dark and dagger in his hand'}]


In [22]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [37]:
TreebankWordDetokenizer().detokenize(["hello", "test"])

'hello test'

In [48]:
f = open("./context.txt","w")
incorrect_generation = 0
total = 0
context = "winter is coming. Jon said "
print(context)
f.write(context)
ITERATION = 3
BOS = "[BOS]"
EOS = "[EOS]"
i = 0
options = []
pred_dialogue = False
while i < ITERATION:
    prev_context_end = len(context)
    context_list = context.split()
    if len(context_list) < 50: # used last 50 words to generate new context
        run_context = TreebankWordDetokenizer().detokenize(context_list)
    else: 
        run_context = TreebankWordDetokenizer().detokenize(context_list[-50:])
    if pred_dialogue:
        run_context += " [BOS]"
    pred = generator(run_context)[0]["generated_text"]
    # print(pred)
    total += 1
    # check [BOS]
    words = pred.split()
    eos_list = []
    bos_list = []
    for j, word in enumerate(words): 
        if word == BOS: 
            bos_list.append(j)
        elif word == EOS:
            eos_list.append(j)
    if len(eos_list) == len(bos_list) and len(eos_list) == 0 and not pred_dialogue:
        # a new context 
        context = pred
        print(context)
        f.write(context)
        print("context continue\n")
    elif len(eos_list) != len(bos_list): 
        print("invalid \n")
        #discard
        incorrect_generation += 1
    else:
        # valid bos eos position
        valid = True 
        for j, value in enumerate(bos_list): 
            if eos_list[j] < value: 
                valid = False
            if j > 0 and value < eos_list[j-1]: 
                # bos bos eos
                valid = False
        if not valid: 
            print("invalid\n")
            incorrect_generation += 1
        else:
            # valid option
            option = TreebankWordDetokenizer().detokenize(words[bos_list[0]+1:eos_list[0]])
            options.append(option)
            pred_dialogue = True
            if len(options) == 3: 
                print("============= select dialogue ============")
                f.write("============= select dialogue ============")
                print("A:{}; \nB:{}; \nC: {}".format(options[0], options[1], options[2]))
                d = {"A": options[0], "B": options[1], "C": options[2]}
                f.write(str(d))
                valid = False
                while not valid:
                    user_input = input()
                    if user_input in ["A", "B", "C"]: 
                        valid = True 
                
                print(d[user_input])
                f.write(d[user_input])
                context += (d[user_input])
                options = []
                i += 1
                pred_dialogue = False

f.close()

winter is coming. Jon said 
invalid 

invalid 

invalid 

invalid 

invalid 

============= select dialogue ============
A:The others are not so much. The Wall has men on it whose mothers were whores.; 
B:My lord, Lady Stark is waiting for me. She ’ s not going to let me leave Winterfell, not for a day or two. So I ’ ll stay a day and a night with him. Have you seen the Wall?; 
C: Yes, my lord. The Stark wordsmiths have done a man a kindness. They've carved his father ’ s brow into the likeness of a direwolf,
Yes, my lord. The Stark wordsmiths have done a man a kindness. They've carved his father ’ s brow into the likeness of a direwolf,
winter is coming. Jon said Yes, my lord. The Stark wordsmiths have done a man a kindness. They've carved his father ’ s brow into the likeness of a direwolf, he said. It ’ s the same as the one Jon saw on the kingsroad, only bigger.The next morning Jon heard Samwell Tarly declare that the middle name was bellied. He had ridden past the end of the world

In [49]:
print(incorrect_generation, total)

15 39
